[Website](https://jiji.ng/cars)

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from tqdm import tqdm

In [24]:
url = "https://jiji.ng/cars"
response = requests.get(url)

path = "C:/Users/HP/Downloads/chromedriver-win64/chromedriver.exe"

service = Service(path)
driver = webdriver.Chrome(service=service)

driver.get(url)

SCROLL_PAUSE_TIME = 0.2

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

for i in tqdm(range(2)):
    # Scroll down to bottom
    scroll_height =  driver.execute_script("return document.body.scrollHeight") *3
    driver.execute_script("window.scrollTo(0,  document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    # if new_height == last_height:
    #    break
    # last_height = new_height

frames = [link.get_attribute('href') for link in driver.find_elements(By.XPATH, "//a[@href][ancestor::*[contains(@class, 'b-list-advert__gallery__item')]]")]

cars = []

for frame in frames: 
    driver.get(frame)
    time.sleep(2)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    specs = {}

    try:
        show_more = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@class='fw-button qa-fw-button fw-button--type-primary-link-like fw-button--size-small']"))
        )
        show_more.click()  
        time.sleep(1)
    except:
        pass

    caption = soup.find_all('div', class_='b-advert-attribute__key')
    header = [cap.text.strip() for cap in caption]

    value = soup.find_all('div', class_='b-advert-attribute__value')
    body = [val.text.strip() for val in value]
    item = [b.replace('\n', '').replace('     ', '').strip() for b in body]

    key, value = header, item
    specs = dict(zip(header, item))

    # price = soup.find('div', class_='qa-advert-price-view-title b-alt-advert-price__text')
    # money = float(price.attrs['content'])
    # specs['Price'] = money

    price_element = driver.find_element(By.XPATH, '//div[@itemprop="price"]')
    price_content = price_element.get_attribute("content")
    specs['Price'] = float(price_content)

    region = soup.find('div', class_='b-advert-info-statistics b-advert-info-statistics--region')
    if region:  
        where = region.text.strip().split(',')[0]
        specs['Location'] = where
    else:
        specs['Location'] = 'Unknown' 

    # attributes = soup.find_all('div', class_='b-advert-attributes--tags')

    # try:
    #     for attribute in attributes: 
    #         attr = attribute.text
    #         xtic = [x.strip() for x in attr.split('\n') if x.strip()]

    #     for x in xtic:
    #         specs[x] = True 
    # except:
    #     pass 

    cars.append(specs)
        
# print(where)

100%|██████████| 2/2 [00:00<00:00,  4.12it/s]


In [26]:
cars

[{'Second Condition': 'No faults, Original parts',
  'Make': 'Hyundai',
  'Model': 'Elantra',
  'Year of Manufacture': '2012',
  'Trim': 'GLS',
  'Body': 'Sedan',
  'Drivetrain': 'Front Wheel',
  'Engine Size': '1800 cc',
  'Number of Cylinders': '4',
  'Horse Power': '150 hp',
  'Color': 'Black',
  'Interior Color': 'Other',
  'Seats': '5',
  'Registered Car': 'Yes',
  'Price': 3950000.0,
  'Location': 'Kaduna'},
 {'Second Condition': 'No faults',
  'Make': 'Lincoln',
  'Model': 'Navigator',
  'Year of Manufacture': '2022',
  'Color': 'Black',
  'Interior Color': 'Beige',
  'Price': 170000000.0,
  'Location': 'Abuja'},
 {'Make': 'Toyota',
  'Model': 'Camry',
  'Year of Manufacture': '2015',
  'Color': 'Silver',
  'Price': 13800000.0,
  'Location': 'Lagos'},
 {'Second Condition': 'No faults',
  'Make': 'Toyota',
  'Model': 'Camry',
  'Year of Manufacture': '2010',
  'Color': 'Red',
  'Registered Car': 'Yes',
  'Price': 6500000.0,
  'Location': 'Lagos'},
 {'Second Condition': 'No faults

In [5]:
url = "https://jiji.ng/jahi/cars/mercedes-benz-c43-2017-silver-frqooo5S6SrxSyo7yKxthLkm.html?page=1&pos=8&cur_pos=8&ads_per_page=23&ads_count=107342&lid=UUkW3P_cAmk_EVop&indexPosition=7"
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html')
features = soup.find_all('div', class_='b-advert-attributes--tags')
price = soup.find('div', class_='qa-advert-price-view-title b-alt-advert-price__text')
region = soup.find('div', class_='b-advert-info-statistics b-advert-info-statistics--region')

caption = soup.find_all('div', class_='b-advert-attribute__key')
header = [cap.text.strip() for cap in caption]

specifications = {} 

value = soup.find_all('div', class_='b-advert-attribute__value')
body = [val.text.strip() for val in value]
w = [b.replace('\n', '').replace('     ', '').strip() for b in body]


key, value = header, w
specifications = dict(zip(header, w))

money = float(price.attrs['content'])
specifications['Price'] = money
where = region.text.strip().split(',')[0]
specifications['Location'] = where

for feature in features: 
    attr = feature.text
    xtic = [x.strip() for x in attr.split('\n') if x.strip()]

for key in xtic:
    specifications[key] = True 
# print(where)
print(specifications)

{'Second Condition': 'No faults', 'Make': 'Mercedes-Benz', 'Model': 'C43', 'Year of Manufacture': '2017', 'Trim': '4Matic Sedan AWD (3.0L V6 9AM)', 'Body': 'Sedan', 'Drivetrain': 'All Wheel', 'Engine Size': '3000 cc', 'Number of Cylinders': '6', 'Color': 'Silver', 'Interior Color': 'Black', 'Registered Car': 'Yes', 'Exchange Possible': 'No', 'Price': 36000000.0, 'Location': 'Abuja', 'Air Conditioning': True, 'AM/FM Radio': True, 'CD Player': True, 'Alloy Wheels': True, 'Airbags': True, 'Blind Spot Monitor': True, 'CarPlay': True, 'Cruise Control': True, 'Electric Mirrors': True, 'Electric Windows': True, 'Front Fog Lamps': True, 'Leather Seats': True, 'Parking Assist': True, 'Parking Sensors': True, 'Sunroof': True}


In [ ]:
url = "https://jiji.ng/jahi/cars/mercedes-benz-c43-2017-silver-frqooo5S6SrxSyo7yKxthLkm.html?page=1&pos=8&cur_pos=8&ads_per_page=23&ads_count=107342&lid=UUkW3P_cAmk_EVop&indexPosition=7"


path = "C:/Users/HP/Downloads/chromedriver-win64/chromedriver.exe"

service = Service(path)
driver = webdriver.Chrome(service=service)

driver.get(url)


# Print the value
print(price_content)

36000000


In [11]:
specifications

{}